this has been tested on champaign & nyc!

In [15]:
#all of my imports
import googlemaps
import numpy as np
from sklearn.cluster import KMeans
from settings import PLACES_API_KEY

API_KEY = PLACES_API_KEY

import numpy as np
from sklearn.cluster import KMeans

def get_nearby_restaurants(latitude, longitude, radius=5000, type="restaurant", num_restaurants=7):
    url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={latitude},{longitude}&radius={radius}&type={type}&key={PLACES_API_KEY}"
    response = requests.get(url)
    data = response.json()
    nameset = set()
    attrs = []
    if "results" in data:
        for place in data["results"]:
            if place["name"] not in nameset:
                attrs.append(
                    [
                        place["name"],
                        place.get("opening_hours"),
                        place["rating"],
                        place["types"],
                        place["photos"],
                    ]
                )
                nameset.add(place["name"])
    return attrs[:num_restaurants]

def recommend_best_restaurants(location, cuisine_preferences, radius=5000, rank_by='prominence', num_clusters=3, num_restaurants=6):
    gmaps = googlemaps.Client(key=API_KEY)
    #keyword = " ".join(cuisine_preferences)

    # Places API search request
    # places_result = gmaps.places_nearby(location=location, radius=radius, keyword=keyword, rank_by=rank_by)
    places_result = []

    for cuisine in cuisine_preferences:
        places_result+= gmaps.places_nearby(location=location, radius=radius, keyword="restaurant " + cuisine, rank_by=rank_by)["results"]

    print(places_result)

    places=places_result
    if len(places) > 0:
        # Latitude & longitude for clustering
        coordinates = [[place['geometry']['location']['lat'], place['geometry']['location']['lng']] for place in places]
        coordinates = np.array(coordinates)

        # Do kmeans clustering
        kmeans = KMeans(n_clusters=num_clusters, random_state=0).fit(coordinates)

        # Find cluster with the highest rated restaurants
        cluster_labels = kmeans.labels_
        max_cluster_label = 0
        max_mean_rating = 0
        for i in range(num_clusters):
            cluster_rating_sum = 0
            cluster_count = 0
            for idx, place in enumerate(places):
                if cluster_labels[idx] == i:
                    cluster_rating_sum += place.get('rating', 0)
                    cluster_count += 1
            if cluster_count > 0:
                mean_rating = cluster_rating_sum / cluster_count
                if mean_rating > max_mean_rating:
                    max_mean_rating = mean_rating
                    max_cluster_label = i

        cluster_center = kmeans.cluster_centers_[max_cluster_label]

        # Calculate distances of restaurants in the highest-rated cluster from its centroid
        distances = []
        for place, label in zip(places, cluster_labels):
            if label == max_cluster_label:
                lat_diff = place['geometry']['location']['lat'] - cluster_center[0]
                lng_diff = place['geometry']['location']['lng'] - cluster_center[1]
                distance = np.linalg.norm([lat_diff, lng_diff])
                distances.append((place, distance))

        # Sort restaurants by rating in descending order
        sorted_restaurants = sorted(distances, key=lambda x: x[0].get('rating', 0), reverse=True)

        # Extract the top six restaurants
        top_restaurants = sorted_restaurants[:num_restaurants]

        # Return ranked list of restaurants
        ranked_restaurants = [restaurant[0] for restaurant in top_restaurants]
    
        if len(ranked_restaurants) < 6:
            ranked_restaurants.append(get_nearby_restaurants(location[0], location[1], num_restaurants=6-len(ranked_restaurants)))
        return ranked_restaurants
    return None

In [16]:
location = (40.11399, -88.225457) #lauren's coordinates
cuisine_preferences = ['Italian', 'Chinese', 'Thai'] 
recommended_restaurants = recommend_best_restaurants(location, cuisine_preferences)
if recommended_restaurants:
    for idx, restaurant in enumerate(recommended_restaurants, start=1):
        print(str(idx)+".", restaurant)
else:
    print("No restaurants found within the specified criteria.")


[{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 40.1158233, 'lng': -88.2389659}, 'viewport': {'northeast': {'lat': 40.11716087989272, 'lng': -88.23750217010726}, 'southwest': {'lat': 40.11446122010727, 'lng': -88.24020182989271}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/restaurant-71.png', 'icon_background_color': '#FF9E67', 'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/restaurant_pinlet', 'name': "Manzella's Italian Patio", 'opening_hours': {'open_now': False}, 'photos': [{'height': 3264, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/112191400593036504334">American Marketing &amp; Publishing</a>'], 'photo_reference': 'ATplDJbK1oqy-wOXD-m3kNtpHFBSFMChRSPA19vbhAGoPhJcTkvBkiil__Vbq5ArcFCsSMazMBX4Jiv3JeANOis_xR9f55YDik10CRmpDizCJeORs_uicPU0alEbZ8mhWICm4crlK1Aki-fQkJ4W0SzYphuMmFfB1YAnAJo5Lpogg52v1g7G', 'width': 4928}], 'place_id': 'ChIJiZIjJknXDIgR8CBqGLrluGE', 'plus_code': {'compound_co